## Consistency Analysis: Evaluating Player Reliability

In basketball, raw scoring totals and efficiency metrics tell only part of the story - consistency can make or break a team’s success. This analysis dives into the game-to-game reliability of players from Partizan Mozzart Bet Belgrade during the 2022/23 season. By examining players with at least 10 active games (`is_playing = True`), I uncover who delivers steady performances and who exhibits scoring patterns, offering insights into dependability and streakiness.

### Objectives

- **Consistency**: Measure variability in points, valuation (e.g., PIR), and plus/minus using standard deviation (SD). Lower SDs highlight players with stable output - key for role players or anchors.
- **Streakiness**: Assess scoring patterns with autocorrelation (lag=1). Positive values indicate streaks (good or bad games cluster), while negative values suggest alternation - crucial for predicting performance trends.
- **Impact**: Contrast consistent contributors with top scorers to balance reliability against offensive firepower.

### Why It Matters

For coaches, analysts, or fans, understanding consistency separates dependable role players from volatile stars. This analysis blends statistical rigor with practical insights.

In [3]:
import pandas as pd

file_path = "../data/partizan_2022_cleaned.csv"

df = pd.read_csv(file_path)

Separate player data from team totals.

In [4]:
# filter 
players = df[df['player'] != 'PARTIZAN MOZZART BET BELGRADE'].copy()
team = df[df['player'] == 'PARTIZAN MOZZART BET BELGRADE'].copy()

Focus analysis on games where players were active.

In [5]:
# filter for games where player actually played
active_players = players[players['is_playing'] == True].copy()

#### Consistency Analysis

Evaluate player consistency across points, valuation, and plus/minus using standard deviation.

In [6]:
# calculate standard deviation of points, valuation, and plus_minus for each player
consistency_stats = active_players.groupby('player').agg({
    'points': 'std',  # scorring consistency
    'valuation': 'std', # overall performance 
    'plus_minus': 'std' # on-court impact
}).rename(columns={
    'points': 'points_sd',
    'valuation': 'valuation_sd',
    'plus_minus': 'plus_minus_sd'
})

# filter for players with at least 10 active games
min_games = 10
consistency_stats = consistency_stats[active_players.groupby('player').size() >= min_games]



print("\nTop 5 Most Consistent Scorers (scorring consistency) (Lowest Points SD):")
print(consistency_stats.sort_values('points_sd')[['points_sd']].head(5))

print("\nTop 5 Most Consistent by Valuation (overall performance) (Lowest Valuation SD):")
print(consistency_stats.sort_values('valuation_sd')[['valuation_sd']].head(5))

print("\nTop 5 Most Consistent by Plus/Minus (on-court impact) (Lowest Plus/Minus SD):")
print(consistency_stats.sort_values('plus_minus_sd')[['plus_minus_sd']].head(5))


Top 5 Most Consistent Scorers (scorring consistency) (Lowest Points SD):
                    points_sd
player                       
Vukcevic Tristan     2.234839
Trifunovic Uros      3.111378
Smailagic Alen       3.926919
Lessort Mathias      4.312145
Papapetrou Ioannis   4.617365

Top 5 Most Consistent by Valuation (overall performance) (Lowest Valuation SD):
                    valuation_sd
player                          
Vukcevic Tristan        3.508232
Trifunovic Uros         4.503407
Smailagic Alen          4.795712
Papapetrou Ioannis      6.004266
Madar Yam               6.588278

Top 5 Most Consistent by Plus/Minus (on-court impact) (Lowest Plus/Minus SD):
                  plus_minus_sd
player                         
Vukcevic Tristan       4.749205
Trifunovic Uros        6.250109
Smailagic Alen         6.836010
Avramovic Aleksa       6.846712
Andjusic Danilo        7.124853


#### Interpretation:

- Vukcevic and Trifunovic have the tightest scoring ranges (SDs ~2–3), suggesting they rarely deviate from their typical output - likely low-scoring bench players with defined, limited roles. Smailagic and Lessort show slightly more variation but still rank high, with Lessort’s presence notable as a probable higher-usage player. Papapetrou rounds out a group of reliable contributors, though his higher SD hints at occasional swings.

- Valuation (e.g., PIR) reflects overall impact beyond scoring. Vukcevic and Trifunovic again lead, indicating steady all-around contributions despite likely low volume. Smailagic’s consistency persists across metrics, while Papapetrou and Madar show moderate variability - still reliable but with more game-to-game fluctuation in their broader impact.

- Plus/minus measures on-court impact. Vukcevic’s low SD (4.75) suggests a stable, likely positive effect when he plays, despite a limited role. Trifunovic and Smailagic follow, reinforcing their reliability. Avramovic and Andjusic, with slightly higher SDs, still maintain consistent influence, hinting at key rotational value.

#### Consistency with Means

Add average values to contextualize consistency metrics.

In [7]:
# add average values to contextualize SDs
consistency_stats_with_means = active_players.groupby('player').agg({
    'points': ['mean', 'std'],
    'valuation': ['mean', 'std'],
    'plus_minus': ['mean', 'std']
})
consistency_stats_with_means.columns = ['points_mean', 'points_sd', 'valuation_mean', 'valuation_sd', 'plus_minus_mean', 'plus_minus_sd']
consistency_stats_with_means = consistency_stats_with_means[active_players.groupby('player').size() >= min_games]



print("\nTop 5 Consistent Scorers with Means:")
print(consistency_stats_with_means.sort_values('points_sd')[['points_mean', 'points_sd']].head(5))
print("\nTop 5 Consistent by Valuation with Means:")
print(consistency_stats_with_means.sort_values('valuation_sd')[['valuation_mean', 'valuation_sd']].head(5))
print("\nTop 5 Consistent by Plus/Minus with Means:")
print(consistency_stats_with_means.sort_values('plus_minus_sd')[['plus_minus_mean', 'plus_minus_sd']].head(5))


Top 5 Consistent Scorers with Means:
                    points_mean  points_sd
player                                    
Vukcevic Tristan       1.071429   2.234839
Trifunovic Uros        1.714286   3.111378
Smailagic Alen         5.400000   3.926919
Lessort Mathias       12.000000   4.312145
Papapetrou Ioannis     5.368421   4.617365

Top 5 Consistent by Valuation with Means:
                    valuation_mean  valuation_sd
player                                          
Vukcevic Tristan          0.000000      3.508232
Trifunovic Uros           1.685714      4.503407
Smailagic Alen            5.033333      4.795712
Papapetrou Ioannis        5.052632      6.004266
Madar Yam                 5.000000      6.588278

Top 5 Consistent by Plus/Minus with Means:
                  plus_minus_mean  plus_minus_sd
player                                          
Vukcevic Tristan         0.642857       4.749205
Trifunovic Uros         -1.628571       6.250109
Smailagic Alen           0.400000  

#### Interpretation:

- Vukcevic (1.07) and Trifunovic (1.71) are low-output players with tight consistency—perfect for predictable bench minutes. ***Smailagic*** and ***Papapetrou*** (~5.4) offer moderate scoring with decent stability, while **Lessort’s** 12-point average with a 4.31 SD stands out as exceptional reliability for a primary scorer, making him a cornerstone.

- Vukcevic’s 0 valuation mean suggests minimal overall impact, but his low SD shows consistency in that limited role. Trifunovic’s low mean aligns with his bench status. Smailagic, Papapetrou, and Madar (~5) contribute steadily across stats, though Madar’s higher SD indicates some inconsistency in his broader performance.

- **Avramovic’s** 3.83 plus/minus mean is a standout, showing a strong positive impact with decent consistency (SD 6.85). Andjusic (1.93) also boosts the team. Vukcevic and Smailagic hover near neutral, while Trifunovic’s negative mean (-1.63) suggests he may play in tougher situations or struggle to influence outcomes.



#### Data Investigation for Autocorrelation

Examine game order to prepare for streakiness analysis.

In [8]:
# investigate data for autocorrelation
print(active_players[['game', 'round', 'phase']].head(10))
print("Unique rounds:", active_players['round'].unique())

      game  round           phase
0  MAD-PAR     35        PLAYOFFS
1  MAD-PAR     39        PLAYOFFS
2  PAR-MAD     32  REGULAR SEASON
3  ULK-PAR     15  REGULAR SEASON
4  PAR-OLY     30  REGULAR SEASON
5  BAR-PAR      9  REGULAR SEASON
6  ZAL-PAR     23  REGULAR SEASON
7  MAD-PAR     10  REGULAR SEASON
8  PAR-BAS     19  REGULAR SEASON
9  TEL-PAR     18  REGULAR SEASON
Unique rounds: [35 39 32 15 30  9 23 10 19 18 20 33 21 13  8 22 11 36 34 16 27 26 12  2
 24  7 29  6 25 31  1  3 14  4 28  5 17 38 37]


Inspect a specific player’s data for autocorrelation validation.

In [9]:
# investigate data for autocorrelation
print(active_players[active_players['player'] == 'Vukcevic Tristan'][['game', 'round', 'phase', 'points']].head(10))

        game  round           phase  points
180  PAR-PAM     11  REGULAR SEASON       0
189  MAD-PAR     36        PLAYOFFS       0
234  PAR-BER     26  REGULAR SEASON       0
243  PAR-RED     12  REGULAR SEASON       2
252  BAS-PAR      2  REGULAR SEASON       3
261  MUN-PAR     24  REGULAR SEASON       0
269  OLY-PAR      7  REGULAR SEASON       0
296  PAR-MAD     37        PLAYOFFS       2
323  BER-PAR      1  REGULAR SEASON       0
341  PAR-IST     14  REGULAR SEASON       0


#### Streakiness Analysis

Assess scoring streakiness using autocorrelation.

In [10]:
# autocorrelation 
# sort by player, phase, and round for autocorrelation
active_players['phase_order'] = active_players['phase'].map({'REGULAR SEASON': 1, 'PLAYOFFS': 2})
active_players = active_players.sort_values(['player', 'phase_order', 'round'])

# calculate autocorrelation of points
autocorr_points = active_players.groupby('player')['points'].apply(
    lambda x: x.autocorr(lag=1) if len(x) >= 10 else float('nan')
).rename('points_autocorr')

autocorr_points = autocorr_points.dropna()



print("\nTop 5 Streaky Scorers (Highest Positive Autocorrelation):")
print(autocorr_points.sort_values(ascending=False).head(5))

print("\nTop 5 Alternating Scorers (Most Negative Autocorrelation):")
print(autocorr_points.sort_values().head(5))


Top 5 Streaky Scorers (Highest Positive Autocorrelation):
player
Trifunovic Uros     0.273616
Vukcevic Tristan    0.140964
Smailagic Alen      0.085139
Madar Yam           0.045379
Andjusic Danilo     0.004133
Name: points_autocorr, dtype: float64

Top 5 Alternating Scorers (Most Negative Autocorrelation):
player
Exum Dante           -0.170288
Lessort Mathias      -0.168292
Papapetrou Ioannis   -0.074709
Leday Zach           -0.044961
Punter Kevin         -0.037450
Name: points_autocorr, dtype: float64


#### Interpretation:

*Highest Positive Autocorrelation*
- Trifunovic (0.27) and Vukcevic (0.14) show mild streakiness - consecutive good or bad scoring games cluster slightly, likely due to their low volume. Smailagic and Madar have weaker positive autocorrelation, suggesting subtle tendencies toward streaks. Andjusic’s near-zero value indicates random scoring patterns.

*Most Negative Autocorrelation*
- Exum and Lessort (~ -0.17) exhibit alternating patterns—high - scoring games often follow low ones, possibly due to matchup changes or usage shifts. Papapetrou, Leday, and Punter have weaker negative values, suggesting mild alternation but mostly independent performances, typical for higher-usage players facing varied defenses.


Examine the autocorrelation of Vukcevic Tristan’s points across games to assess whether his scoring in one game is predictive of his scoring in the next game, indicating consistency or momentum in performance.

In [11]:
# validate autocorrelation for Vukcevic Tristan
vukcevic = active_players[active_players['player'] == 'Vukcevic Tristan']

print(vukcevic[['round', 'phase', 'points']])
print("Autocorr:", vukcevic['points'].autocorr(lag=1))

     round           phase  points
323      1  REGULAR SEASON       0
252      2  REGULAR SEASON       3
350      4  REGULAR SEASON       8
368      5  REGULAR SEASON       0
269      7  REGULAR SEASON       0
180     11  REGULAR SEASON       0
243     12  REGULAR SEASON       2
341     14  REGULAR SEASON       0
377     17  REGULAR SEASON       0
261     24  REGULAR SEASON       0
234     26  REGULAR SEASON       0
359     28  REGULAR SEASON       0
189     36        PLAYOFFS       0
296     37        PLAYOFFS       2
Autocorr: 0.140964264666771


### Interpretation  

#### Scoring Pattern  
- **Vukcevic Tristan**’s points are highly variable, with many games at 0 points and peaks at 8, 3, and 2 points.  
- He played 14 games (after filtering for `active_players` with at least 10 games and removing rows with missing `points_previous_game`), with most in the regular season and two in the playoffs.  

#### Autocorrelation  
- The autocorrelation of 0.141 indicates a weak positive relationship between consecutive games’ points.  
- A value close to 0 suggests that **Vukcevic Tristan**’s scoring in one game does not strongly predict his scoring in the next, implying inconsistency or situational performance (e.g., limited minutes or matchup dependency).  

#### Context  
- As a role player, his scoring is likely secondary to other contributions like defense, which may explain the low and inconsistent points.

Evaluate player consistency by calculating the mean and standard deviation of points, valuation, and plus-minus, and include points autocorrelation to assess scoring momentum. Identify the top 5 most consistent scorers based on the lowest standard deviation in points.

In [12]:
# combined stats
consistency_full = active_players.groupby('player').agg({
    'points': ['mean', 'std'],
    'valuation': ['mean', 'std'],
    'plus_minus': ['mean', 'std']
})
consistency_full.columns = ['points_mean', 'points_sd', 'valuation_mean', 'valuation_sd', 'plus_minus_mean', 'plus_minus_sd']
consistency_full = consistency_full[active_players.groupby('player').size() >= min_games]
consistency_full['points_autocorr'] = autocorr_points



print("\nTop 5 Consistent Scorers with Autocorr:")
print(consistency_full.sort_values('points_sd')[['points_mean', 'points_sd', 'points_autocorr']].head(5))


Top 5 Consistent Scorers with Autocorr:
                    points_mean  points_sd  points_autocorr
player                                                     
Vukcevic Tristan       1.071429   2.234839         0.140964
Trifunovic Uros        1.714286   3.111378         0.273616
Smailagic Alen         5.400000   3.926919         0.085139
Lessort Mathias       12.000000   4.312145        -0.168292
Papapetrou Ioannis     5.368421   4.617365        -0.074709


#### Explanation

- **Points mean (`points_mean`)**: Average points per game for players with at least 10 active games (`is_playing = True`). Vukcevic Tristan (1.07) and Trifunovic Uros (1.71) are low-usage players, while Lessort Mathias (12.00) stands out as a higher-scoring contributor.

- **Standard deviation (`points_sd`)**: Measures scoring consistency - lower values indicate less variation game-to-game. Vukcevic’s 2.23 SD is the tightest, reflecting minimal fluctuation (e.g., 0–3 points), likely due to a limited role. Lessort’s 4.31 SD, though higher, is impressive for a 12-point average, showing reliable output for a key player.

- **Autocorrelation (`points_autocorr`)**: Indicates scoring streakiness (lag=1, game-to-game). Positive values suggest streaks (e.g., Trifunovic’s 0.27 shows mild consistency in low scoring), while negative values indicate alternation (e.g., Lessort’s -0.17 suggests high games follow low ones). Most values are near zero, implying Partizan players’ scoring is generally independent of prior games.

#### Insights
- **Role players**: Vukcevic and Trifunovic exemplify bench consistency - low means and SDs with slight streakiness, ideal for predictable, small contributions.
- **Key contributor**: Lessort Mathias balances higher scoring with solid consistency (SD 4.31) but alternates performance, possibly due to matchup or usage swings.
- **Versatility**: Smailagic and Papapetrou offer moderate scoring (≈5.4) with decent stability, though Papapetrou’s slight negative autocorrelation hints at inconsistency.

This analysis highlights dependable performers, blending statistical rigor (SD, autocorrelation) with practical insights - valuable for team strategy or player evaluation.

Evaluate player performance by calculating the mean and standard deviation of points, valuation, and plus-minus, and include points autocorrelation to assess scoring momentum. Identify the top 5 players with the highest average points to understand who the leading scorers are, along with their consistency and scoring trends.

In [13]:
# combined stats
consistency_full = active_players.groupby('player').agg({
    'points': ['mean', 'std'],
    'valuation': ['mean', 'std'],
    'plus_minus': ['mean', 'std']
})
consistency_full.columns = ['points_mean', 'points_sd', 'valuation_mean', 'valuation_sd', 'plus_minus_mean', 'plus_minus_sd']
consistency_full = consistency_full[active_players.groupby('player').size() >= min_games]
consistency_full['points_autocorr'] = autocorr_points



print("\nTop 5 Scorers by Average Points with Autocorr:")
print(consistency_full.sort_values('points_mean', ascending=False)[['points_mean', 'points_sd', 'points_autocorr']].head(5))


Top 5 Scorers by Average Points with Autocorr:
                 points_mean  points_sd  points_autocorr
player                                                  
Punter Kevin       16.081081   6.529329        -0.037450
Exum Dante         13.184211   6.150611        -0.170288
Lessort Mathias    12.000000   4.312145        -0.168292
Leday Zach         11.435897   6.294504        -0.044961
Nunnally James      9.162162   6.020860        -0.037105


#### Explanation

- **Points mean (`points_mean`)**: Average points per game for players with at least 10 active games. Punter Kevin leads with 16.08, marking him as Partizan’s top scorer, followed by Exum Dante (13.18) and Lessort Mathias (12.00).

- **Standard deviation (`points_sd`)**: Reflects scoring variability. Lessort’s 4.31 SD is the lowest, showing tight consistency for a double-digit scorer. Punter (6.53) and Exum (6.15) have higher SDs, typical for high-usage players with bigger scoring swings (e.g., 10–22 points).

- **Autocorrelation (`points_autocorr`)**: Measures scoring patterns. All values are near zero or negative, with Exum (-0.17) and Lessort (-0.17) showing mild alternation (e.g., a high-scoring game followed by a lower one). Punter’s -0.04 suggests near-random game-to-game output.

#### Insights
- **Star scorers**: Punter and Exum drive the offense with high averages, though their higher SDs indicate volatility—expected for primary options.
- **Balanced impact**: Lessort Mathias stands out, pairing a solid 12-point average with the lowest SD (4.31), making him a reliable scoring anchor.
- **Randomness**: Near-zero or negative autocorrelation across the board suggests these top scorers don’t ride hot streaks - performance varies independently, likely due to defensive attention or role shifts.

This highlights key offensive contributors, combining scoring volume with consistency metrics - ideal for evaluating impact players.